### Import

In [26]:
import os
import re  
import fnmatch 
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import scipy.stats as stats
from scipy.stats import chi2_contingency

from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.ensemble import RandomForestClassifier

### Path

In [7]:
parquet_dir = './Data/parquet/'
output_file = './Data/'

### Read Population Labels

In [8]:
pop_labels = pd.read_csv(parquet_dir + 'igsr_samples.tsv', sep='\t')
pop_labels.head(3)

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
0,HG00271,male,SAME123417,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1,HG00276,female,SAME123424,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2,HG00288,female,SAME1839246,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."


### Read, Clean, and Merge SNPs

In [9]:
def read_and_clean_chromosome_files(folder_path):

    for chr_num in range(1, 23):
        
        file_path = f"{folder_path}/chr{chr_num}.parquet"
        df = pd.read_parquet(file_path, engine='pyarrow')
        
        missing_percentage = df.isnull().mean() * 100
        cols_to_keep = missing_percentage[missing_percentage <= 5].index
        cleaned_df = df[cols_to_keep]
        
        if chr_num == 1:
            final = cleaned_df.copy()
        else:
            final = final.merge(cleaned_df, on='Person_ID')
                
    return final

In [10]:
result_df = read_and_clean_chromosome_files(parquet_dir)
result_df = result_df.set_index('Person_ID')

In [11]:
result_df.head(3)

,rs3094315,rs3131972,rs12124819,rs11240777,rs6681049,rs4970383,rs4475691,rs7537756,rs13302982,rs1110052,...,rs10451,rs715586,rs8137951,rs2301584,rs756638,rs3810648,rs2285395,rs13056621,rs5771007,rs3888396
Person_ID,,,,,,,,,,,,,,,,,,,,,
HG00096,1,1,0,1,2,0,0,0,2,1,...,0,1,0,0,0,0,0,1,0,1
HG00097,2,2,0,0,2,1,1,1,2,0,...,0,0,0,0,0,0,0,1,0,0
HG00099,1,1,0,1,2,0,0,0,2,2,...,1,0,1,0,0,0,0,2,0,0


### Outlier Detection

Identify all values that are outside the range of 0 to 2 (inclusive), and set those values to NaN, which represents missing data in pandas DataFrames

In [12]:
def clean_snp_data(df):
    
    mask = (df < 0) | (df > 2)
    df[mask] = np.nan
    
    return df

result_df = clean_snp_data(result_df)

### Remove SNPs with Missing Values > 5% 

Remove: If a sample (row) or SNP (column) has a high percentage of missing values, you might consider removing it entirely from the dataset.

In [13]:
threshold = 0.05 * result_df.shape[0]  
result_df = result_df.dropna(axis='columns', thresh=threshold)

### Impute Missing Values with Mode

Impute: For SNPs with a small percentage of missing data, you can impute missing values. Common strategies include replacing missing values with the median or mode. In SNP data, using the most common genotype (0, 1, or 2) among the observed values for that SNP can be a sensible approach.

In [14]:
def selective_imputation(df):
    
    missing_columns = df.columns[df.isnull().any()].tolist()

    imputer = SimpleImputer(strategy='most_frequent')
    
    if missing_columns:
        df[missing_columns] = imputer.fit_transform(df[missing_columns])
    
    return df

result_df = selective_imputation(result_df)

### Minor Allele Frequency (MAF)

SNPs with a very low MAF are often excluded since they may not contribute much information for ancestry prediction and can lead to model overfitting. A common threshold is to exclude SNPs with a MAF less than 5%.

In [15]:
def filter_snps_by_maf_A(df, maf_threshold=0.05):

    maf_dict = {}
    total_alleles = 2 * len(df)
    
    for column in df.columns:
        
        counts = df[column].value_counts()
        
        freq_minor_allele = min((2 * counts.get(2, 0) + counts.get(1, 0)) / total_alleles,
                                (2 * counts.get(0, 0) + counts.get(1, 0)) / total_alleles)
        
        maf_dict[column] = freq_minor_allele
    
    columns_to_keep = [column for column, maf in maf_dict.items() if maf >= maf_threshold]
    
    filtered_df = df[columns_to_keep]
    
    return filtered_df

In [16]:
def filter_snps_by_maf_B(dataframe, maf_threshold=0.05):

    maf_dict = {}
    to_drop  = []

    for snp in dataframe.columns:

        genotype_counts = dataframe[snp].value_counts()
        
        n_AA = genotype_counts.get(0, 0)
        n_Aa = genotype_counts.get(1, 0)
        n_aa = genotype_counts.get(2, 0)
        
        total_alleles = 2 * (n_AA + n_Aa + n_aa)
        freq_A = (2 * n_AA + n_Aa) / total_alleles
        freq_a = 1 - freq_A  
        
        maf = min(freq_A, freq_a)
        maf_dict[snp] = maf
        
        if maf < maf_threshold:
            to_drop.append(snp)
    
    filtered_df = dataframe.drop(columns=to_drop)

    return filtered_df

In [17]:
result_df = filter_snps_by_maf_B(result_df, maf_threshold=0.05)
print(result_df.shape)

(2504, 823111)


### Hardy-Weinberg Equilibrium (HWE)

Deviations from HWE can indicate problems such as genotyping errors. While checking for HWE is more common in association studies, you might decide to perform this check and exclude SNPs not in equilibrium if you suspect data quality issues.

In [18]:
def filter_snps_by_hwe_A(df, hwe_p_threshold=0.01):

    columns_to_keep = []
    
    for column in df.columns:
        
        genotype_counts = df[column].value_counts().sort_index()

        for genotype in [0, 1, 2]:  
            if genotype not in genotype_counts.index:
                genotype_counts.at[genotype] = 0

        p = (2 * genotype_counts[0] + genotype_counts[1]) / (2 * sum(genotype_counts.values)) 
        q = 1 - p 

        expected_counts = pd.Series({0: (p ** 2)  * sum(genotype_counts.values),  
                                     1: 2 * p * q * sum(genotype_counts.values), 
                                     2: (q ** 2)  * sum(genotype_counts.values)})

        chi2, p_value = stats.chisquare(f_obs=genotype_counts, f_exp=expected_counts)

        if p_value >= hwe_p_threshold:
            columns_to_keep.append(column)
            
    filtered_df = df[columns_to_keep]
    
    return filtered_df

In [19]:
def filter_snps_by_hwe_B(dataframe, alpha=0.01):

    hwe_results = {}
    
    for snp in dataframe.columns:
        
        genotype_counts = dataframe[snp].value_counts()
        
        obs_AA = genotype_counts.get(0, 0)
        obs_Aa = genotype_counts.get(1, 0)
        obs_aa = genotype_counts.get(2, 0)
        
        total_alleles = 2 * (obs_AA + obs_Aa + obs_aa)
        p = (2 * obs_AA + obs_Aa) / total_alleles 
        q = 1 - p  
        
        exp_AA = (p ** 2)  * (obs_AA + obs_Aa + obs_aa)
        exp_Aa = 2 * p * q * (obs_AA + obs_Aa + obs_aa)
        exp_aa = (q ** 2)  * (obs_AA + obs_Aa + obs_aa)
        
        observed = np.array([obs_AA, obs_Aa, obs_aa])
        expected = np.array([exp_AA, exp_Aa, exp_aa])
        chi2, p_value = chi2_contingency([observed, expected], correction=False)[:2] 
        
        is_in_equilibrium = p_value >= alpha
        
        hwe_results[snp] = (chi2, p_value, is_in_equilibrium)
    
    return hwe_results

In [20]:
def filter_snps_from_hwe_B(dataframe, hwe_results):

    snps_to_keep = [snp for snp, (_, _, is_in_equilibrium) in hwe_results.items() if is_in_equilibrium]
    filtered_dataframe = dataframe[snps_to_keep]
    
    return filtered_dataframe

In [21]:
hwe_results = filter_snps_by_hwe_B(result_df, alpha=0.01)
result_df = filter_snps_from_hwe_B(result_df, hwe_results)
print(result_df.shape)

(2504, 499504)


### Variance Threshold Filtering 

Variance Threshold: Remove SNPs that have low variance across samples, as they might not be informative.

In [22]:
def variance_threshold_filtering(df, p):
    
    threshold = p * (1 - p)
    selector = VarianceThreshold(threshold)

    df_transformed = selector.fit_transform(df)
    df_transformed = pd.DataFrame(df_transformed, columns=df.columns[selector.get_support()], index= df.index)

    kept_columns = df.columns[selector.get_support()]
    dropped_columns = df.columns[~selector.get_support()]
    
    return df_transformed, kept_columns, dropped_columns

In [23]:
result_df, _, _ = variance_threshold_filtering(result_df, 0.85)
print(result_df.shape)

(2504, 477372)


In [24]:
result_df.head(3)

,rs12124819,rs6681049,rs4970383,rs4475691,rs7537756,rs3748597,rs28391282,rs2340592,rs1891910,rs3128117,...,rs739365,rs5770992,rs2040487,rs9628187,rs6010063,rs10451,rs715586,rs8137951,rs2301584,rs3810648
Person_ID,,,,,,,,,,,,,,,,,,,,,
HG00096,0.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,2.0,...,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
HG00097,0.0,2.0,1.0,1.0,1.0,2.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HG00099,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0


### Preparing Population Codes

In [25]:
pop_labels = pop_labels[pop_labels['Sample name'].isin(result_df.index)]
pop_labels = pop_labels[['Sample name', 'Superpopulation code']]
pop_labels.rename(columns={"Sample name": "Person_ID", "Superpopulation code": "labels"}, inplace=True)

### Principal Component Analysis (PCA)

In [28]:
def apply_pca(df, n_components=None, explained_variance=None):

    if n_components is not None:
        pca = PCA(n_components=n_components)
    elif explained_variance is not None:
        pca = PCA(n_components=explained_variance)
    else:
        pca = PCA()
    
    principal_components = pca.fit_transform(df)
    
    col_names = [f'PC{i+1}' for i in range(principal_components.shape[1])]
    pca_df = pd.DataFrame(data=principal_components, columns=col_names, index= df.index)
    
    return pca_df

In [29]:
pca_data = apply_pca(result_df, n_components=1000, explained_variance=0.90)  

In [30]:
pca_data.head(3)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC991,PC992,PC993,PC994,PC995,PC996,PC997,PC998,PC999,PC1000
Person_ID,,,,,,,,,,,,,,,,,,,,,
HG00096,-38.639026,-90.794450,23.263929,-35.948275,1.161187,9.309609,-0.001401,2.061211,2.417782,6.942762,...,-5.468540,-7.901774,9.770670,-11.258240,-7.951597,-2.844346,5.893723,2.953390,-4.820451,-3.457041
HG00097,-38.439010,-93.450054,22.201406,-36.684623,-0.139637,6.492132,-0.724865,-1.474823,0.837887,10.620088,...,3.764506,-4.264768,-13.994740,-14.554333,10.144423,-4.262681,-4.673796,11.506335,-5.590803,-9.564501
HG00099,-38.349643,-91.178250,24.402136,-35.676430,1.074924,14.443116,1.314256,-5.785633,8.926218,4.024488,...,-2.241154,-11.742524,5.602238,2.376374,-14.044337,4.720400,3.871844,-0.231161,0.315130,-8.066297


In [31]:
print(pca_data.shape)

(2504, 1000)


### Singular Value Decomposition (SVD)

In [32]:
def apply_svd_sklearn(df, n_components=100):

    svd = TruncatedSVD(n_components=n_components)
    svd_fit = svd.fit_transform(df)
    
    col_names = [f'SVD{i+1}' for i in range(n_components)]
    svd_df = pd.DataFrame(data=svd_fit, columns=col_names, index= df.index)
    
    return svd_df

In [33]:
def apply_svd_numpy(df, n_components=100):

    centered_data = df - df.mean()
    
    U, s, Vt = np.linalg.svd(centered_data, full_matrices=False)  
    
    U_reduced  = U[:, :n_components]
    s_reduced  = s[:n_components]
    Vt_reduced = Vt[:n_components, :]

    reduced_data = np.dot(U_reduced, np.diag(s_reduced))
    
    col_names = [f'SVD{i+1}' for i in range(n_components)]
    svd_df = pd.DataFrame(data=reduced_data, columns=col_names, index= df.index)
    
    return svd_df

In [34]:
svd_data = apply_svd_sklearn(result_df, n_components=1000)

In [35]:
svd_data_df = apply_svd_numpy(result_df, n_components=1000)  

In [37]:
svd_data.head(3)

,SVD1,SVD2,SVD3,SVD4,SVD5,SVD6,SVD7,SVD8,SVD9,SVD10,...,SVD991,SVD992,SVD993,SVD994,SVD995,SVD996,SVD997,SVD998,SVD999,SVD1000
Person_ID,,,,,,,,,,,,,,,,,,,,,
HG00096,603.046621,-39.278318,-90.627046,23.277228,-36.099755,1.153453,9.316863,0.012396,2.155326,2.370304,...,3.156259,-1.474766,1.161947,0.780569,-1.222766,11.323725,-3.306233,11.323746,-2.614192,-3.830227
HG00097,601.038719,-39.184792,-93.150561,22.195952,-36.778369,-0.131148,6.513195,-0.761343,-1.570738,1.100527,...,7.461926,0.411498,-4.711832,1.720158,-6.858940,2.205314,-5.848622,-4.742616,-0.772183,7.926935
HG00099,605.540751,-38.887557,-91.187782,24.442167,-35.910246,1.037098,14.451912,1.300676,-6.059226,8.863826,...,2.322019,-10.555962,8.663292,6.594597,-6.732870,-12.189297,-2.027765,0.143378,-7.920362,2.626753


### Random Forest

In [40]:
def apply_rf_feature_selection(df, labels_df, num_features=None, importance_threshold=None):
    
    merged_df = df.merge(labels_df, left_index=True, right_on='Person_ID')
    
    data = merged_df.drop(columns=['Person_ID', 'labels'])
    labels = merged_df['labels']

    rf = RandomForestClassifier(random_state=42)
    rf.fit(data, labels)

    feature_importances = rf.feature_importances_

    if importance_threshold is not None:
        features_to_keep = df.columns[feature_importances >= importance_threshold]
    elif num_features is not None:
        sorted_features = df.columns[np.argsort(-feature_importances)]
        features_to_keep = sorted_features[:num_features]
    else:
        features_to_keep = df.columns

    reduced_df = df[features_to_keep]
    
    return reduced_df, rf

In [41]:
reduced_df, rf = apply_rf_feature_selection(result_df, pop_labels, num_features=1000, importance_threshold=None)

In [43]:
reduced_df.head(3)

,rs7128655,rs1999047,rs10224196,rs1390151,rs1144628,rs17242334,rs11862829,rs12516794,rs6573875,rs2824119,...,rs11171949,rs10850526,rs2252523,rs9914327,rs8107859,rs12207656,rs7729096,rs7570835,rs577737,rs953062
Person_ID,,,,,,,,,,,,,,,,,,,,,
HG00096,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0
HG00097,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,1.0,...,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0
HG00099,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0
